In [ ]:
from florence_utils import run_inference
from llama_utils import call_llama_exp8
from ram_utils import load_tags_from_json,get_tags_for_id

c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
c:\Users\redti\anaconda3\envs\cvproject\Lib\site-packages\fairscale\experimental\nn\offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


[2025/01/19 20:51:52] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\redti/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\redti/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=

In [ ]:
import os
import json

# Directory containing the images
image_directory = "data/image"
tags_json_file =  "data/tags.json"
tags_dict = load_tags_from_json(tags_json_file)
output_data = []

# Loop through each image file and perform the inference for each task
for i in range(1, 1005):
    try:
    # Construct the image path
        img_path =  f"{image_directory}/AMBER_{i}.jpg"

        # Run inference with each task type
        image_tags = get_tags_for_id(i, tags_dict)

        object_detection_info = run_inference(img_path, '<OD>')

        ocr_info = run_inference(img_path, '<OCR>')

        caption = run_inference(img_path, '<DETAILED_CAPTION>')

        final_response = call_llama_exp8(ocr_info, object_detection_info,image_tags,caption)
        print(final_response)
        print(f"Response for image {i}: {final_response}")
        # Append the result for this image
        output_data.append({"id": i, "response": final_response})
    except:
        print("Issue on ID: "+str(i))

# Save the results to a JSON file
output_file = "generated_captions/8_detailed_caption_with_grounding.json"
with open(output_file, "w") as file:
    json.dump(output_data, file, indent=4)

print(f"Evaluation results saved to {output_file}")
